In [3]:
# Import all the functions you need
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path

from neuropy import core
from neuropy.io import (optitrackio,
                        dlcio,
                        )
from neuropy.io.neuroscopeio import NeuroscopeIO
from neuropy.io.binarysignalio import BinarysignalIO 
from neuropy.io.miniscopeio import MiniscopeIO


In [4]:
# Define a class for a typical recording or set of recordings
class ProcessData:
    def __init__(self, basepath):
        basepath = Path(basepath)
        self.basepath = basepath
        xml_files = sorted(basepath.glob("*.xml"))
        assert len(xml_files) == 1, "Found more than one .xml file"
        
        fp = xml_files[0].with_suffix("")
        self.filePrefix = fp
        
        self.recinfo = NeuroscopeIO(xml_files[0])
        self.eeginfo = BinarysignalIO(self.recinfo.eeg_filename,
                                     n_channels=self.recinfo.n_channels,
                                     sampling_rate=self.recinfo.eeg_sampling_rate,
                                     )
        
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.recinfo.source_file.name})"
    
def Rat698recall():
    basepath='/data/Working/Trace_FC/Recording_Rats/Rat698/2021_06_30_recall'
    return ProcessData(basepath)

In [5]:
sess = Rat698recall()
print(sess.recinfo)

filename: /data/Working/Trace_FC/Recording_Rats/Rat698/2021_06_30_recall/continuous_combined.xml 
# channels: 35
sampling rate: 30000
lfp Srate (downsampled): 1250


In [7]:
sess.eeginfo.n_frames

7240022

# Import miniscope data and timestamps

First, run the session through the minian pipeline and save your S, A, and C variables to numpy arrays in a "minian" folder somewhere in the session folder

Second, designate any *corrupted files* by creating a "corrupted_videos.csv" file in the folder where your timeStamps.csv file lives. List the index to each corrupted file separated by commas. Even if you have concatenated all your videos from different recordings together in minian this file you should still put this in the file for each recording with a corrupted file.

In [4]:
# Now, load in all the video timestamps and minian data.
sess.miniscope = MiniscopeIO(sess.basepath)
sess.miniscope.load_all_timestamps()
sess.miniscope.load_minian()

Eliminating timestamps from corrupted video6 in 13_00_45 folder.


In [5]:
# Check that # timestamps matches neural data
print(sess.miniscope.times_all.shape)
print(sess.miniscope.minian['S'].shape)

(27367, 4)
(80, 27367)


# Get timestamps for neural data 

In [6]:
sess.basepath

PosixPath('/data/Working/Trace_FC/Recording_Rats/Rat698/2021_06_30_recall')

In [ ]:
# NRK start on 10/6/2021
# 1) have timestamps for everything now, interpolate everything to ephys times! Or maybe just use timestamps?  
# 2) Would be good to pull in my .eeg file and see what types of things I can analyze from it.  Also figure out
# a way to import the mua from my 3-wire electrode.


# Get TTLs to OE for synchronization

In [7]:
# NRK note - check in code if timestamps between OE and optitrack match up well, otherwise chuck them and go by timestamps!

# Plan for alignment
designate each recording session in `epochs`, then have an `align_by` flag that will either align by TTL or by timestamps or, in the case of oe and position, align by theta v speed.  Also, for some sessions I have an input from the miniscope to Intan that I can use.

Since some epochs will have a TTL at the start but not later on after a disconnect, you will need to either a) fake a "disconnect" event and only use the TTL if there is an off-on transition after the disconnect, or b) just search for the closet TTL PRIOR to each timestamp to figure out how to align it.  Function would look identify a range of times and then flag them as either "align_by_timestamps" or "align_by_TTL" or "align_by_theta" (position to oe only) or "MS_to_oe_TTL".

# Options for alignment

## 1) TTL - digital or analog?

## 2) Timestamps - currently not accurate enough

## 3) theta as a backup

# Read in DLC pos data and sync with Optitrack timestamps